In [4]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("Distributions")
#Pkg.add("Ipopt")
#Pkg.add("Juniper")
using Ipopt
using JuMP
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper
include("run_optimization.jl")
include("scenario_generation.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


sample_cf (generic function with 1 method)

In [5]:
#generate the scenarios
S = 4 # Small right now, but change it later
seasons = rand(["winter", "spring", "summer", "fall"], S)
times = rand(1:24, S)

cf = [sample_cf(s, t) for (s, t) in zip(seasons, times)]
lf = [sample_load(s, t) for (s, t) in zip(seasons, times)]

4-element Vector{Float64}:
 0.5340882120498772
 0.9283991901410902
 0.8497562525920399
 0.496386333173591

In [9]:
#run the optimization model
data_path = "DATA/"
baseMVA = 10;

# Run the grid search
step_size = 0.25  # Define the step size for the grid search

function grid_search(step_size, data_path, baseMVA, cf, lf)
    results = DataFrame(alpha=Float64[], beta=Float64[], gamma=Float64[], objective_cost=Float64[])

    # Iterate over possible values of alpha and beta
    for alpha in 0:step_size:1
        for beta in 0:step_size:(1 - alpha)
            gamma = 1 - alpha - beta
            if gamma >= 0  # Ensure the constraint is satisfied
                # Calculate the objective cost
                cost = run_optimization(data_path, baseMVA, alpha, beta, gamma, cf, lf)
                # Append the results to the DataFrame
                push!(results, (alpha, beta, gamma, cost))
            end
        end
    end

    return results
end

# Run the grid search
results = grid_search(step_size)

LoadError: MethodError: no method matching grid_search(::Float64)

[0mClosest candidates are:
[0m  grid_search(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[9]:8[24m[39m


In [7]:
output_file = "grid_search_results.csv"
CSV.write(output_file, results)

LoadError: UndefVarError: `results` not defined